## The following tables/csv files will be created after running this notebook:
 
 

       1. Product - columns [ 'uniq_id','Product_Name','Manufacturer','Price','average_review_rating',
                           'amazon_category_and_sub_category','description']
                           
       2. Review - columns ['uniq_id', 'customer_reviews']
       
       3. AlsoBought - columns ['uniq_id', customers_who_bought_this_item_also_bought]

       4. BuyAfterView - columns ['uniq_id', items_customers_buy_after_viewing_this_item]
      
       5. Customer - columns ['customer_id','name','username','password']
       
       6. Order - columns ['order_id', 'uniq_id', 'customer_id'] 
       

      Other possible enhancements:
       - add image to product 
       - mapping product with customer in a meaningful way (recommendation/association - ML components?)
       
       

In [1]:
import pandas as pd
import re 
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

#pip install names
import names

In [2]:
# Huan
def strip(lst):
    """
    Return a list of string without whitespace
    """
    return [i.strip() for i in lst]

def review(text):
    """
    Parse and return the reviews 
    """
    pattern= re.compile(r"^(By\n)|^[\d]+|^(ecx)|^(g-ecx)|^(if\(ue\))")
    reviews=[]
    for i in text:
        if pattern.search(i) == None:
            reviews.append(i)
    
    return reviews

def recommend_unstack(df):
    """
    Parse and return the recommendation links
    """
    unstack = pd.DataFrame(df.iloc[:,1].str.split('|').tolist(), index=df.uniq_id).stack()
    unstack = unstack.reset_index([0,df.columns[0]])
    unstack.columns = df.columns
    unstack.fillna('',inplace=True)
    return unstack

def review_unstack(df):
    """
    Parse and return the review data
    """
    df['customer_reviews'].fillna('',inplace=True)
    df['customer_reviews'] = df['customer_reviews'].apply(lambda x: x.split('//')).apply(strip)
    df['customer_reviews'] = df['customer_reviews'].apply(review)
    unstack = df['customer_reviews'].apply(pd.Series)
    unstack = unstack.merge(df, 
                            right_index = True, 
                            left_index = True).drop(["customer_reviews"], axis = 1)
    unstack = unstack.melt(id_vars = 'uniq_id',
                           value_name = "customer_reviews").drop(columns='variable')
    unstack.dropna(inplace=True)
    
    return unstack

# Teija - Please copy your code here and run it with the "data" dataframe 

In [3]:
# Teija





In [4]:
# Rob
data = pd.read_csv('amazon_co-ecommerce_sample.csv')
rob = pd.read_csv('RobsDataCleaned.csv',encoding='latin-1')
rob = rob.iloc[:,:-1].dropna()

In [5]:
data = rob.merge(data,on='uniq_id')

In [6]:
data.drop(columns=['product_name_y','manufacturer_y','price','number_of_reviews'],inplace=True)
data.rename(columns={"product_name_x": "Product_Name", "manufacturer_x": "Manufacturer"}, inplace=True)

In [7]:
data.head()

,uniq_id,Product_Name,Manufacturer,Price,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,items_customers_buy_after_viewing_this_item,customer_reviews
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,Unknown,49.88,5.0 out of 5 stars,Figures & Playsets > Science Fiction & Fantasy,http://www.amazon.co.uk/Resident-Evil-Kennedy-...,Product Description From Square Enix! From the...,http://www.amazon.co.uk/Resident-Evil-Revelati...,Good figure // 5.0 // 30 Jan. 2014 // By\n ...
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,George,24.99,4.3 out of 5 stars,Characters & Brands > Disney > Toys,http://www.amazon.co.uk/Disney-Frozen-Elsas-Mu...,"LET IT go, let it go, cant hold it back any mo...",NaN,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...
2,0030f5965cbd07a138147425b9a0156c,Disney The Lion King Childrens 24 Piece Simba ...,King International,6.53,5.0 out of 5 stars,Characters & Brands > Disney > Toys,http://www.amazon.co.uk/Ravensburger | http://...,NaN,http://www.amazon.co.uk/Disney-King-4-In-1-Puz...,Five Stars // 5.0 // 2 Mar. 2015 // By\n \n...
3,0034d77b68130bba71aca6e733c7e1ac,Pokemon Black and White Voice Activated Talkin...,Takara Tomy,30.23,5.0 out of 5 stars,Characters & Brands > Tomy,http://www.amazon.co.uk/Pokemon-Oshawott-Plush...,Pokemon Black and White Voice Activated Talkin...,NaN,great // 5.0 // 10 Jun. 2013 // By\n \n ...
4,0039295c7fcefba10d5fc333eff806d3,LEGO City Airport 60022: Cargo Terminal,LEGO,199.18,4.7 out of 5 stars,NaN,http://www.amazon.co.uk/LEGO-City-Airport-6002...,Product Description Get busy loading the cargo...,http://www.amazon.co.uk/LEGO-60095-Explorers-E...,Lego City Cargo Terminal // 4.0 // 18 Nov. 201...


In [8]:
#Product Table
product=data[['uniq_id','Product_Name',
              'Manufacturer','Price',
              'average_review_rating',
              'amazon_category_and_sub_category',
              'description']]

In [9]:
product.to_csv('product.csv')
product.head()

,uniq_id,Product_Name,Manufacturer,Price,average_review_rating,amazon_category_and_sub_category,description
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,Unknown,49.88,5.0 out of 5 stars,Figures & Playsets > Science Fiction & Fantasy,Product Description From Square Enix! From the...
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,George,24.99,4.3 out of 5 stars,Characters & Brands > Disney > Toys,"LET IT go, let it go, cant hold it back any mo..."
2,0030f5965cbd07a138147425b9a0156c,Disney The Lion King Childrens 24 Piece Simba ...,King International,6.53,5.0 out of 5 stars,Characters & Brands > Disney > Toys,NaN
3,0034d77b68130bba71aca6e733c7e1ac,Pokemon Black and White Voice Activated Talkin...,Takara Tomy,30.23,5.0 out of 5 stars,Characters & Brands > Tomy,Pokemon Black and White Voice Activated Talkin...
4,0039295c7fcefba10d5fc333eff806d3,LEGO City Airport 60022: Cargo Terminal,LEGO,199.18,4.7 out of 5 stars,NaN,Product Description Get busy loading the cargo...


In [10]:
AlsoBought = data[['uniq_id','customers_who_bought_this_item_also_bought']]
BuyAfterView = data[['uniq_id','items_customers_buy_after_viewing_this_item']]
reviews = data[['uniq_id','customer_reviews']]
AlsoBought.fillna('',inplace=True)
BuyAfterView.fillna('',inplace=True)
reviews.fillna('',inplace=True)

/Users/huanchen/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [11]:
#Review Table
reviews = review_unstack(reviews)
reviews.to_csv('reviews.csv')
reviews.head()

/Users/huanchen/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/huanchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/huanchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing

,uniq_id,customer_reviews
0,0016eb63fa6c7a5e8930bc7732b13116,Good figure
1,00182f9b6081e58fea1fe76bc13eb022,Five Stars
2,0030f5965cbd07a138147425b9a0156c,Five Stars
3,0034d77b68130bba71aca6e733c7e1ac,great
4,0039295c7fcefba10d5fc333eff806d3,Lego City Cargo Terminal


In [12]:
# Recommendation Table for customers_who_bought_this_item_also_bought
AlsoBought = recommend_unstack(AlsoBought)
AlsoBought.to_csv('AlsoBought.csv')
AlsoBought.head()

,uniq_id,customers_who_bought_this_item_also_bought
0,0016eb63fa6c7a5e8930bc7732b13116,http://www.amazon.co.uk/Resident-Evil-Kennedy-...
1,0016eb63fa6c7a5e8930bc7732b13116,http://www.amazon.co.uk/Resident-Evil-Sheva-A...
2,0016eb63fa6c7a5e8930bc7732b13116,http://www.amazon.co.uk/Batman-Arkham-Origins...
3,0016eb63fa6c7a5e8930bc7732b13116,http://www.amazon.co.uk/Star-Images-Clarke-Ac...
4,0016eb63fa6c7a5e8930bc7732b13116,http://www.amazon.co.uk/Square-Enix-Tomb-Raid...


In [13]:
# Recommendation Table for items_customers_buy_after_viewing_this_item
BuyAfterView = recommend_unstack(BuyAfterView)
BuyAfterView.to_csv('BuyAfterView.csv')
BuyAfterView.head()

,uniq_id,items_customers_buy_after_viewing_this_item
0,0016eb63fa6c7a5e8930bc7732b13116,http://www.amazon.co.uk/Resident-Evil-Revelati...
1,0016eb63fa6c7a5e8930bc7732b13116,http://www.amazon.co.uk/Resident-Evil-Kennedy...
2,00182f9b6081e58fea1fe76bc13eb022,
3,0030f5965cbd07a138147425b9a0156c,http://www.amazon.co.uk/Disney-King-4-In-1-Puz...
4,0030f5965cbd07a138147425b9a0156c,http://www.amazon.co.uk/Disney-King-Childrens...


In [14]:
num_customer = 50
customer = pd.DataFrame(columns=['customer_id','name','username','password'])
customer['customer_id'] = ['c'+str(i) for i in range(num_customer)]
customer['name'] = [names.get_full_name() for i in range(num_customer)]
customer['username'] = customer['name'].apply(lambda x: x.replace(' ',str(len(x))))
customer['password'] = 'backrow'
customer.to_csv('customer.csv')
customer.head()

,customer_id,name,username,password
0,c0,Pauline Sutton,Pauline14Sutton,backrow
1,c1,Vanessa Box,Vanessa11Box,backrow
2,c2,Diana Benham,Diana12Benham,backrow
3,c3,Ester Sasser,Ester12Sasser,backrow
4,c4,Christel Nash,Christel13Nash,backrow


In [15]:
num_order = 100
Order = pd.DataFrame(columns=['order_id', 'uniq_id', 'customer_id'])
Order['order_id'] = ['o'+str(i) for i in range(num_order)]
Order['uniq_id'] = [list(product['uniq_id'].sample())[0] for i in range(num_order)]
Order['customer_id'] = [list(customer['customer_id'].sample())[0] for i in range(num_order)]
Order.to_csv('order.csv')

In [16]:
Order.head()

,order_id,uniq_id,customer_id
0,o0,4fdafda2815e05b2f2f87a1fe0928922,c41
1,o1,e98815a63cb7ecbae96c0bf2e3dc7cf9,c40
2,o2,4bc2a8b7ba48c5d36d70aa2c2edf780e,c39
3,o3,034aae3907a3030b9c2d64410576f0f9,c16
4,o4,066e1663446e3513220aba24b41a9dc1,c3
